# SVM localization for NB-IoT

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/NBIOT_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAVAT1,BMAXMT1,BMAXMT2,BMAXMT3,BMBACT1,BMBACT2,BMBAKT1,...,BMZLTT1,BMZLTT3,TBTS1T1,TBTS3T1,TBTS3T2,TBTS5T1,TBTS5T2,TBTS5T3,TBTS6T1,TBTS6T2
0,1701853012,49.227134,16.575806,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1701853013,49.227133,16.575806,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1701853014,49.227133,16.575805,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1701853015,49.227132,16.575804,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1701853016,49.227131,16.575803,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28672,1701951192,49.230384,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28673,1701951194,49.230385,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28674,1701951196,49.230386,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28675,1701951198,49.230387,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [4]:
# Scaling X and y
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, shuffle=True, test_size=0.2)

## SVM

In [7]:
from sklearn.svm import SVR

In [8]:
regressor_lat = SVR(kernel='rbf')
regressor_lat.fit(X_train, y_train[:,0])

SVR()

In [9]:
regressor_lon = SVR(kernel='rbf')
regressor_lon.fit(X_train, y_train[:,1])

SVR()

In [10]:
y_pred_lat = regressor_lat.predict(X_test)
y_pred_lon = regressor_lon.predict(X_test)

In [11]:
y_pred_lat

array([-0.69617173,  0.77361057,  0.10093735, ...,  0.77100378,
       -1.96316578,  1.66576261])

In [12]:
y_pred_lon

array([-0.30886502,  0.87860789,  0.39556422, ...,  0.53811259,
       -0.2410384 , -0.36681188])

In [13]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [14]:
# R^2 score
score_lat = r2_score(y_test[:,0], y_pred_lat)

print("R^2 lat - {}%".format(round(score_lat, 5) *100))

R^2 lat - 98.07000000000001%


In [15]:
score_lon = r2_score(y_test[:,1], y_pred_lon)

print("R^2 lon - {}%".format(round(score_lon, 5) *100))

R^2 lon - 98.84100000000001%


In [16]:
score_combine = (score_lat + score_lon)/2

print("R^2 combine - {}%".format(round(score_combine, 5) *100))

R^2 combine - 98.455%


## MSE and RMSE

In [17]:
y_test_fin = sc_y.inverse_transform(y_test)
y_test_fin

array([[49.186634, 16.59501 ],
       [49.209336, 16.642385],
       [49.199535, 16.626282],
       ...,
       [49.211574, 16.627767],
       [49.163797, 16.59734 ],
       [49.227039, 16.595299]])

In [18]:
y_pred_join = np.stack((y_pred_lat, y_pred_lon), axis=1)
y_pred_join

array([[-0.69617173, -0.30886502],
       [ 0.77361057,  0.87860789],
       [ 0.10093735,  0.39556422],
       ...,
       [ 0.77100378,  0.53811259],
       [-1.96316578, -0.2410384 ],
       [ 1.66576261, -0.36681188]])

In [19]:
y_pred_fin = sc_y.inverse_transform(y_pred_join)
y_pred_fin

array([[49.18595707, 16.59788012],
       [49.21025042, 16.64026693],
       [49.19913212, 16.6230247 ],
       ...,
       [49.21020733, 16.62811296],
       [49.16501552, 16.60030119],
       [49.22499639, 16.59581171]])

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mse_svm = mean_squared_error(y_test_fin, y_pred_fin)
rmse_svm = mean_squared_error(y_test_fin, y_pred_fin, squared = False)

print(mse_svm)
print(rmse_svm)

1.0342244540789392e-05
0.0031130453670295266


In [22]:
# Just to confirm R^2 calculation before and after de-scaling the coordinates
score_r2 = r2_score(y_test_fin, y_pred_fin)
score_r2*100

98.45537709571417

## Distance error

In [23]:
from math import acos, sin, cos, radians

# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist


In [24]:
geo_dist_svm = geo_distance(y_test_fin, y_pred_fin)
geo_dist_svm

array([0.22175752, 0.1844241 , 0.24087164, ..., 0.15403017, 0.25439135,
       0.2301595 ])

In [25]:
geo_dist_svm_m = [i * 1000 for i in geo_dist_svm]

In [26]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_svm = Average(geo_dist_svm_m)

# Printing average of the list
print("Average SVM error [m] =", round(average_error_svm, 5))
print("Max SVM error [m] = ", np.max(geo_dist_svm_m))
print("Min SVM error [m] = ", np.min(geo_dist_svm_m))

Average SVM error [m] = 280.95731
Max SVM error [m] =  3807.681172944703
Min SVM error [m] =  2.941462623916587


# Cross check

In [27]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/NBIOT_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAVAT1,BMAXMT1,BMAXMT2,BMAXMT3,BMBACT1,BMBACT2,BMBAKT1,...,BMZLTT1,BMZLTT3,TBTS1T1,TBTS3T1,TBTS3T2,TBTS5T1,TBTS5T2,TBTS5T3,TBTS6T1,TBTS6T2
0,1701439521,49.227657,16.575590,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1701439522,49.227656,16.575592,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1701439523,49.227656,16.575593,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1701439524,49.227654,16.575595,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1701439525,49.227653,16.575596,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7349,1701447326,49.197588,16.608551,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7350,1701447327,49.197582,16.608546,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7351,1701447328,49.197578,16.608546,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7352,1701447329,49.197579,16.608549,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]
y_check_ori = cross_check_df.iloc[:, 1:3]

In [29]:
# Scaling cross check data

sc_X_cross = StandardScaler()
sc_y_cross = StandardScaler()
X_check = sc_X_cross.fit_transform(X_check)
y_check = sc_y_cross.fit_transform(y_check)

In [30]:
y_cross_lat = regressor_lat.predict(X_check)
y_cross_lon = regressor_lon.predict(X_check)

In [31]:
# R^2 score
score_lat_cross = r2_score(y_check[:,0], y_cross_lat)
print("R^2 lat cross check - {}%".format(round(score_lat_cross, 5) *100))

score_lon_cross = r2_score(y_check[:,1], y_cross_lon)
print("R^2 lon cross check - {}%".format(round(score_lon_cross, 5) *100))

score_combine_cross = (score_lat_cross + score_lon_cross)/2
print("R^2 combine cross check - {}%".format(round(score_combine_cross, 5) *100))

R^2 lat cross check - 81.95599999999999%
R^2 lon cross check - 64.694%
R^2 combine cross check - 73.32499999999999%


## MSE and RMSE

In [32]:
y_cross_join = np.stack((y_cross_lat, y_cross_lon), axis=1)
y_cross_join

array([[ 1.69138604, -0.84296719],
       [ 1.73941511, -0.86297098],
       [ 1.76211021, -0.8724531 ],
       ...,
       [ 0.04540567, -0.08725523],
       [ 0.0433971 , -0.0877707 ],
       [ 0.03934593, -0.08879285]])

In [33]:
y_cross_fin = sc_y_cross.inverse_transform(y_cross_join)
y_cross_fin

array([[49.22806261, 16.59415651],
       [49.22896946, 16.59373634],
       [49.22939797, 16.59353718],
       ...,
       [49.19698437, 16.61002979],
       [49.19694644, 16.61001896],
       [49.19686995, 16.60999749]])

In [34]:
y_cross_fin.shape

(7354, 2)

In [35]:
mse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin)
rmse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin, squared = False)

print(mse_svm_cross)
print(rmse_svm_cross)

0.00011004751559019144
0.01025057632403768


## Distance cross check

In [36]:
y_check_arr = y_check_ori.to_numpy()

In [37]:
geo_dist_svm_cross = geo_distance(y_check_arr, y_cross_fin)
geo_dist_svm_cross

array([1.348982  , 1.3256296 , 1.31733708, ..., 0.12641564, 0.12788831,
       0.13153799])

In [38]:
geo_dist_svm_cross_m = [i * 1000 for i in geo_dist_svm_cross]
geo_dist_svm_cross_m

[1348.982002386136,
 1325.6295955921532,
 1317.3370775158833,
 1317.226177137888,
 1317.1707405535751,
 1316.9716710039586,
 1316.8444371343226,
 1324.8971642366168,
 1316.4689225400423,
 1324.5240041128736,
 1324.3673554807156,
 1324.2230141080925,
 1324.1508443278701,
 1324.0786706139866,
 1324.0065031765237,
 1323.9466465535754,
 1323.8146297508117,
 1323.6826270166832,
 1323.5506349506516,
 1323.490822403583,
 1323.4186569609958,
 1315.1701230481171,
 1315.1147745500448,
 1323.299058097396,
 1323.2392784507786,
 1323.2640774219838,
 1323.3486499293294,
 1323.3610688796546,
 1323.385940482389,
 1323.4108456687675,
 1323.4357878419362,
 1323.5329267415548,
 1315.434741176728,
 1323.6300925577486,
 1323.64261100279,
 1323.7272989017622,
 1323.8119881911662,
 1323.8841458086727,
 1323.9562960897763,
 1323.9688386798596,
 1324.028449246232,
 1324.1006018742362,
 1324.1602201216601,
 1324.2920051082367,
 1334.670709198662,
 1362.6846918690753,
 1362.7577176990885,
 1362.8299228821459,
 1

In [39]:
average_error_svm_cross = Average(geo_dist_svm_cross_m)

# Printing average of the list
print("Average SVM error cross check [m] =", round(average_error_svm_cross, 5))
print("Max SVM error cross check [m] = ", np.max(geo_dist_svm_cross_m))
print("Min SVM error cross check [m] = ", np.min(geo_dist_svm_cross_m))

Average SVM error cross check [m] = 1094.58948
Max SVM error cross check [m] =  4807.132142089661
Min SVM error cross check [m] =  19.208862021159238
